In [1]:
import mysql.connector as mysql
import pandas as pd

In [6]:
db_conn = mysql.connect(host = '10.4.19.215', user = 'sugengw07', password= 'sgwi2341',
database = 'mpninfo',port = '3306')

bruto21_kueri = '''select sum(nominal) as jumlah from netto2021
where ket  <> 'SPMKP'
and datebayar <= curdate() ;'''
bruto21 = pd.read_sql(bruto21_kueri,db_conn)['jumlah'].sum()


In [7]:
str(bruto21)[:3]

'957'

In [ ]:
data.datebayar = pd.to_datetime(data.datebayar)
data = data.set_index('datebayar').sort_index()

In [ ]:
datasdokto = data.loc['2021-01-01':'2021-10-31']

In [ ]:
datasdokto.nominal.sum()

In [ ]:
bruto = data[data['ket']=='MPN']

In [ ]:
bruto.nominal.sum()

In [ ]:
ppmpkmq = '''SELECT A.NPWP,
       A.KPP,
       A.CABANG,
       A.NAMA,
       A.KDMAP,
       A.KDBAYAR,
       MASA,
       A.TAHUN,
       TAHUNBAYAR,
       BULANBAYAR,
       DATEBAYAR,
       NOMINAL,
       CASE
         WHEN SUBSTRING(A.KDMAP,1,5) IN ('41131','41111') THEN 'PPM'
         WHEN TAHUNBAYAR = A.TAHUN THEN 'PPM'
         WHEN TAHUNBAYAR - A.TAHUN IN (0,1) AND A.KDMAP IN ('411125','411126') AND A.KDBAYAR IN ('200','199','310','320','390','500','501') THEN 'PPM'
         WHEN (TAHUNBAYAR - A.TAHUN IN (0,1) AND MASA = 12) THEN 'PPM'
         WHEN A.TAHUN > TAHUNBAYAR THEN 'PPM'
         ELSE 'PKM'
       END FLAG_PPM_PKM,
       NIPPJ,
       ntpn,
       nosk,
       E.SEKTOR,
       URAIAN,
       F.NAMA AS NAMA_KLU,
       F.SEKTOR AS SEKTOR_KLU
FROM MPN A
  LEFT JOIN WP B
         ON A.NPWP = B.NPWP
        AND A.KPP = B.KPP
        AND A.CABANG = B.CABANG
  LEFT JOIN MAP E
         ON A.KDMAP = E.KDMAP
        AND A.KDBAYAR = E.KDBAYAR
  LEFT JOIN KLU F ON B.KLU = F.KODE
WHERE TAHUNBAYAR = 2021;'''

In [27]:
db_conn = mysql.connect(host = '10.4.19.215', user = 'sugengw07', password= 'sgwi2341',
database = 'mpninfo',port = '3306')
#with open(r'sql\PPM_PKM_2021.sql','r') as query:
    #penerimaan = query.read()
ppmpkm = pd.read_sql(ppmpkmq,db_conn)
mf = pd.read_excel(r'MFWP/MFWP.xlsx',usecols=[
'NPWP','NAMA_WP','NAMA_AR','SEKSI'], dtype={'NPWP':'str'},engine='openpyxl')

In [ ]:
ppmpkm.DATEBAYAR = pd.to_datetime(ppmpkm.DATEBAYAR )

In [ ]:
ppmpkm=ppmpkm.set_index('DATEBAYAR').sort_index()

In [ ]:
ppmpkm.BULANBAYAR = ppmpkm.BULANBAYAR.astype('str')

In [33]:
slice = pd.merge(ppmpkm,mf, left_on='NPWP',right_on='NPWP',how='left')

In [34]:
slice = slice.filter(['NAMA_WP','BULANBAYAR','KDMAP','NAMA','NOMINAL'])

In [35]:
slice = slice.groupby(['NAMA_WP']).sum().reset_index().sort_values(by = 'NOMINAL',ascending=False)

In [36]:
slice

,NAMA_WP,BULANBAYAR,NOMINAL
259,FASTRATA BUANA,78761,9.236137e+12
22,AKUR PRATAMA,22581,3.849663e+12
673,PARIT PADANG GLOBAL,8158650,3.691865e+12
604,MITSUBISHI MOTORS KRAMA YUDHA SALES INDONESIA,6799,1.249639e+12
571,MENSA BINASUKSES,2058903,9.402395e+11
...,...,...,...
202,DANIEL ROOSENO,20,2.000000e+05
82,AZIZAH NURAINI,18,2.000000e+05
14,AFIYAH HASANAH UTAMA,10,1.000000e+05
332,HASIL BUMI MAKMUR,6,1.000000e+05


In [25]:
slice.to_clipboard()

In [ ]:
sdoktober = ppmpkm.loc['2021-01-01':'2021-10-31']

In [ ]:
ppm = sdoktober[sdoktober['FLAG_PPM_PKM']=='PPM']
pkm = sdoktober[sdoktober['FLAG_PPM_PKM']=='PKM']

In [ ]:
print('{:,.0f}'.format(ppm.NOMINAL.sum()))

In [ ]:
print('{:,.0f}'.format(pkm.NOMINAL.sum()))

In [ ]:
spmkp = pd.read_sql('select * from spmkp;',db_conn)

In [ ]:
spmkp.tahun = spmkp.tahun.astype('str')

In [ ]:
spmkp = spmkp[(spmkp['bulan']<11) & (spmkp['tahun']=='2021')]

In [ ]:
print('{:,.0f}'.format(spmkp.nominal.sum()))

In [ ]:
ppm = ppm.filter(['NAMA_SEKSI','NAMA_AR','NPWP_FULL', 'NAMA','NTPN','NOMINAL'])
pkm = pkm.filter(['NAMA_SEKSI','NAMA_AR','NPWP_FULL', 'NAMA','NTPN','NOMINAL'])

In [ ]:
ppm= ppm.groupby(['NAMA_AR','NPWP_FULL']).sum().reset_index()
pkm= pkm.groupby(['NAMA_AR','NPWP_FULL']).sum().reset_index()

In [ ]:
ppm.to_excel('PPM_Oktober.xlsx',index=False)

In [ ]:
pkm.to_excel('PKM_Oktober.xlsx',index=False)

In [ ]:
print('{:,.0f}'.format(pkm.NOMINAL.sum()))

In [ ]:
smo = data[data['BULANBAYAR'].isin([6,7,8,9,10])]
print('{:,.0f}'.format(smo.NOMINAL.sum()))

In [ ]:
mpn.nominal.sum()

In [ ]:
data.to_excel('PPMPKM_Juni_Oktober2021.xlsx',index=False)

In [ ]:
bruto21_kueri = '''select sum(nominal) as jumlah from mpn
where datebayar between '2020-12-31' and curdate() ;'''
bruto20_kueri = '''select sum(nominal) as jumlah from mpn
where datebayar > '2019-12-31' and
datebayar <= date_sub(curdate(),interval 1 year);'''

bruto21 = pd.read_sql(bruto21_kueri,db_conn)
bruto20 = pd.read_sql(bruto20_kueri,db_conn)

In [ ]:
bruto21.subtract(bruto20)

In [ ]:
(bruto21.subtract(bruto20))/bruto20['jumlah'].sum()

In [ ]:

with open(r'sql\mpnspmpbkspmkp.sql','r') as query:
    penerimaan = query.read()
data21 = pd.read_sql(penerimaan,db_conn)
data21['NPWP_FULL'] = data.npwp+data.kpp+data.cabang
mf = mfwp()
data21 = pd.merge(data21,mf, left_on='NPWP_FULL',right_on='NPWP', how='left')
data21.drop('nama',axis=1,inplace=True)
    
with open(r'sql\mpnspmpbkspmkp.sql','r') as query:
    penerimaan = query.read()
data = pd.read_sql(penerimaan,db_conn)
data['NPWP_FULL'] = data.npwp+data.kpp+data.cabang
mf = mfwp()
data = pd.merge(data,mf, left_on='NPWP_FULL',right_on='NPWP', how='left')
data.drop('nama',axis=1,inplace=True)
    
def mfwp():
    mf = pd.read_excel(r'MFWP/MFWP+klu.xlsx',usecols=[
        'NPWP','NAMA_WP','NAMA_AR','SEKSI','NAMA_KLU'], dtype={'NPWP':'str'},engine='openpyxl')
    return mf

In [ ]:
mpn21 = netto21()
mpn20 = netto20()